In [1]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import time 
import os
import sys

import pandas as pd
import numpy as np

import matplotlib.pylab as plt
import lightgbm as lgb

from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import (
    train_test_split,
    TimeSeriesSplit,
    KFold,
    StratifiedKFold,
    GroupKFold,
    StratifiedGroupKFold,
)

plt.style.use('seaborn-white')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import random


from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

/var/folders/0j/qw0wf1sj6gj4qpttd2789y8c0000gn/T/ipykernel_4150/645041300.py:27: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')
2023-01-01 11:03:50.272135: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
blosum_train_0 = pd.read_csv(os.path.join(sys.path[0],"Analysis/blosum50_train.csv"), index_col=False)
print(blosum_train_0.shape)

blosum_test_0 = pd.read_csv(os.path.join(sys.path[0],"Analysis/blosum50_test.csv"), index_col=False)
print(blosum_test_0.shape)

(87052, 1233)
(21535, 1233)


In [3]:
ifeature_train_0 = pd.read_csv(os.path.join(sys.path[0],"Analysis/ifeature_train.csv"), index_col=False)
print(ifeature_train_0.shape)

ifeature_test_0 = pd.read_csv(os.path.join(sys.path[0],"Analysis/ifeature_test.csv"), index_col=False)
print(ifeature_test_0.shape)


ifeature_train_0 = ifeature_train_0.drop(columns=["Unnamed: 0"])
ifeature_test_0 = ifeature_test_0.drop(columns=["Unnamed: 0"])

(87052, 2065)
(21535, 2065)


In [4]:
def columns_arrange_blosum(df):

    df = df.drop(columns=["Peptide","Y", "MHC", "Peptide_L", "Peptide_ID"])

    cols = list(df.columns.values) #Make a list of all of the columns in the df
    cols.pop(cols.index('y')) #Remove b from list
    df = df[cols+['y']]
    cols = list(df.columns.values)
    cols.pop(cols.index('Pair_ID'))
    df = df[['Pair_ID']+cols]
    print(df.shape)
    
    return df

In [5]:
blosum_train = columns_arrange_blosum(blosum_train_0)
blosum_test = columns_arrange_blosum(blosum_test_0)

(87052, 1228)
(21535, 1228)


In [6]:
ifeature_train = columns_arrange_blosum(ifeature_train_0)
ifeature_test = columns_arrange_blosum(ifeature_test_0)

(87052, 2059)
(21535, 2059)


In [7]:
blosum_train.y.value_counts()

0    49258
1    37794
Name: y, dtype: int64

In [8]:
ifeature_test.y.value_counts()

0    12846
1     8689
Name: y, dtype: int64

In [9]:
blosum_train_all = blosum_train.drop(columns=["Pair_ID", "MHC_ID"])
blosum_test_all = blosum_test.drop(columns=["Pair_ID", "MHC_ID"])
ifeature_train_all = ifeature_train.drop(columns=["Pair_ID", "MHC_ID"])
ifeature_test_all = ifeature_test.drop(columns=["Pair_ID", "MHC_ID"])

In [10]:
def get_X_y(df, scaler):


    X = df.iloc[:,:-1]
    X = scaler.transform(X)
    # X = tf.convert_to_tensor(X)
    y = df.iloc[:,-1]
    # y = tf.convert_to_tensor(y)

    print(X.shape, y.shape)
    
    print("===end===")
    return X, y

In [11]:
scaler = MinMaxScaler(feature_range=(0, 1)).fit(blosum_train_all.drop(columns=["y"]))
X_blosum_train, y_blosum_train = get_X_y(blosum_train_all, scaler)
X_blosum_test, y_blosum_test = get_X_y(blosum_test_all, scaler)

(87052, 1225) (87052,)
===end===
(21535, 1225) (21535,)
===end===


In [12]:
### blosum feature reshape for RNN
# timesteps = 25

X_blosum_train_RNN = X_blosum_train.reshape(87052, 25, 49)
X_blosum_test_RNN = X_blosum_test.reshape(21535, 25, 49)

In [13]:
scaler = MinMaxScaler(feature_range=(0, 1)).fit(ifeature_train_all.drop(columns=["y"]))
X_ifeature_train, y_ifeature_train = get_X_y(ifeature_train_all, scaler)
X_ifeature_test, y_ifeature_test = get_X_y(ifeature_test_all, scaler)

(87052, 2056) (87052,)
===end===
(21535, 2056) (21535,)
===end===


In [14]:
del blosum_train_0, blosum_test_0, ifeature_train_0, ifeature_test_0

# Model

In [15]:
def test_evaluation(model, X_test, y_test):
    pred_label_1 = model.predict(X_test, batch_size=128, verbose=2)
    pred_label_number_1 = np.argmax(pred_label_1, axis=1)
    test_labels = y_test
    report = classification_report(test_labels, pred_label_number_1, digits = 3)
    print(report)
    print("AUROC:", roc_auc_score(test_labels, pred_label_number_1))

    return pred_label_1, pred_label_number_1, report

## 1. combined (RNN+DNN)

In [25]:
def individual_blosum(X_blosum, X_blosum_t, X_ifeature, X_ifeature_t, Y, Y_t, neuron1, neuron2, epoch1, epoch2):

    # Model1
    inputs = keras.Input(shape=X_blosum.shape[1:])
    x = layers.SimpleRNN(neuron1, activation="relu", kernel_regularizer=regularizers.l2(0.01), return_sequences=True)(inputs)
    x = layers.SimpleRNN(245, activation="relu", kernel_regularizer=regularizers.l2(0.01))(inputs)
    outputs = layers.Dense(2, name = 'my_layer')(x)

    model = keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    # if use CategoricalCrossentropy, then the output should be one-hot encoded
    optimizer = keras.optimizers.Adamax(learning_rate=0.001),
    metrics=["accuracy"] # the metrics keras will keep tracking during the training
    )

    model.fit(X_blosum, Y, batch_size=128, epochs=epoch1, verbose=2)

    
    model = keras.Model(inputs=inputs, outputs=[layer.output for layer in model.layers])
    features = model.predict(X_blosum, batch_size=128)
    features_test = model.predict(X_blosum_t, batch_size=128)

    base_output = features[1]
    print("train latent output: ", base_output.shape)
    base_output_t = features_test[1]
    print("test latent output: ", base_output_t.shape)
    
    
    # Model2
    X_concat = np.concatenate((base_output, X_ifeature), axis=1)
    X_concat_t = np.concatenate((base_output_t, X_ifeature_t), axis=1)

    inputs = keras.Input(shape=X_concat.shape[1])
    x = layers.Dense(neuron2, activation="relu", kernel_regularizer=regularizers.l2(0.01))(inputs)
    x = layers.Dropout(0.5)(x)
    # x = layers.Dense(neuron3, activation="relu", kernel_regularizer=regularizers.l2(0.01))(inputs)
    outputs = layers.Dense(2)(x)


    model = keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    # if use CategoricalCrossentropy, then the output should be one-hot encoded
    optimizer = keras.optimizers.Adamax(learning_rate=0.001),
    metrics=["accuracy"] # the metrics keras will keep tracking during the training
    )

    model.fit(X_concat, Y, batch_size=128, epochs = epoch2, verbose=2)

    pred_label_1 = model.predict(X_concat_t, batch_size=128, verbose=2)
    pred_label_number_1 = np.argmax(pred_label_1, axis=1)
    test_labels = Y_t

    report = classification_report(test_labels, pred_label_number_1, digits = 3)
    print(report)
    print("AUROC:", roc_auc_score(test_labels, pred_label_number_1))
    print("test latent output: ", base_output_t.shape)


    return model, pred_label_1, pred_label_number_1, report

In [26]:
model_combine, pred_label_combined, pred_label_number_combined, report_combined = individual_blosum(X_blosum_train_RNN, X_blosum_test_RNN, X_ifeature_train, X_ifeature_test, y_ifeature_train , y_ifeature_test, 60, 257, 20, 20)

Epoch 1/20
681/681 - 13s - loss: 0.8176 - accuracy: 0.7159 - 13s/epoch - 20ms/step
Epoch 2/20
681/681 - 12s - loss: 0.5958 - accuracy: 0.7467 - 12s/epoch - 18ms/step
Epoch 3/20
681/681 - 14s - loss: 0.5491 - accuracy: 0.7558 - 14s/epoch - 20ms/step
Epoch 4/20
681/681 - 14s - loss: 0.5268 - accuracy: 0.7611 - 14s/epoch - 20ms/step
Epoch 5/20
681/681 - 16s - loss: 0.5109 - accuracy: 0.7658 - 16s/epoch - 23ms/step
Epoch 6/20
681/681 - 15s - loss: 0.5023 - accuracy: 0.7698 - 15s/epoch - 22ms/step
Epoch 7/20
681/681 - 14s - loss: 0.4964 - accuracy: 0.7734 - 14s/epoch - 20ms/step
Epoch 8/20
681/681 - 14s - loss: 0.4902 - accuracy: 0.7764 - 14s/epoch - 20ms/step
Epoch 9/20
681/681 - 14s - loss: 0.4862 - accuracy: 0.7788 - 14s/epoch - 20ms/step
Epoch 10/20
681/681 - 14s - loss: 0.4824 - accuracy: 0.7819 - 14s/epoch - 20ms/step
Epoch 11/20
681/681 - 14s - loss: 0.4772 - accuracy: 0.7855 - 14s/epoch - 20ms/step
Epoch 12/20
681/681 - 14s - loss: 0.4709 - accuracy: 0.7887 - 14s/epoch - 20ms/step
E

In [34]:
report_combined = classification_report(y_ifeature_test, pred_label_number_combined, digits = 3)
print(report_combined)
print("AUROC:", roc_auc_score(y_ifeature_test, pred_label_number_combined))

              precision    recall  f1-score   support

           0      0.832     0.728     0.776     12846
           1      0.661     0.783     0.716      8689

    accuracy                          0.750     21535
   macro avg      0.746     0.755     0.746     21535
weighted avg      0.763     0.750     0.752     21535

AUROC: 0.7553409461305001


In [35]:
state="train1_combined_model"


lines = report_combined
auroc = str(0.7553409461305001)

with open('Result/class_report_%s.txt'%(state), 'w') as f:
    for line in lines:
        f.write(line)
    f.write(auroc)

In [14]:
# Format 2
def individual_blosum(X, Y, neuron, epoch):

  ### build layers
  model = keras.Sequential(
      [
          keras.Input(shape=X.shape[1:]),
          layers.SimpleRNN(neuron, activation="relu", kernel_regularizer=regularizers.l2(0.01), return_sequences=True),
          layers.SimpleRNN(245, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
          # layers.Dropout(0.5),
        #   layers.SimpleRNN(neuron, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
          # layers.Dense(neuron, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
          layers.Dense(2),
      ]
  )

  print(model.summary())


  
  ### compile the model configuration
  model.compile(
      loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      # if use CategoricalCrossentropy, then the output should be one-hot encoded
      optimizer = keras.optimizers.Adamax(learning_rate=0.001),
      metrics=["accuracy"] # the metrics keras will keep tracking during the training (tf.keras.metrics.AUC())
  )

  ### fit the data
  # callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True), validation_data=(X_test, Y_test),callbacks = [callback]
  history = model.fit(X, Y, batch_size=128, epochs=epoch, verbose=2)

  return model

In [15]:
model_blosum = individual_blosum(X_blosum_train_RNN, y_blosum_train, 60, 20)

2023-01-01 10:40:21.916207: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 25, 60)            6600      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 245)               74970     
                                                                 
 dense (Dense)               (None, 2)                 492       
                                                                 
Total params: 82,062
Trainable params: 82,062
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
681/681 - 17s - loss: 1.0349 - accuracy: 0.7057 - 17s/epoch - 25ms/step
Epoch 2/20
681/681 - 15s - loss: 0.6837 - accuracy: 0.7346 - 15s/epoch - 23ms/step
Epoch 3/20
681/681 - 15s - loss: 0.6095 - accuracy: 0.7412 - 15s/epoch - 23ms/step
Epoch 4/20
681/681 - 16s - loss: 0.5737 - accuracy: 0.7484 - 1

In [18]:
model_blosum

In [30]:
blosum_all_label, blosum_all_label_number, blosum_report = test_evaluation(model_blosum, X_blosum_test_RNN, y_blosum_test)

169/169 - 2s - 2s/epoch - 10ms/step
              precision    recall  f1-score   support

           0      0.787     0.801     0.794     12846
           1      0.697     0.679     0.688      8689

    accuracy                          0.752     21535
   macro avg      0.742     0.740     0.741     21535
weighted avg      0.751     0.752     0.751     21535

AUROC: 0.7398678130603946


In [31]:
state="train1_blosum"


lines = blosum_report
auroc = str(0.7398678130603946)

with open('Result/class_report_%s.txt'%(state), 'w') as f:
    for line in lines:
        f.write(line)
    f.write(auroc)

### iFeature

In [20]:
# Format 2
def individual_ifeature(X, Y, neuron, epoch):

  ### build layers
  model = keras.Sequential(
      [
          keras.Input(shape=X.shape[1]),
          layers.Dense(neuron, activation="relu", kernel_regularizer=regularizers.l2(0.01)),  
          layers.Dropout(0.5),
        #   layers.Dense(neuron, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
        #   layers.Dropout(0.5),
        #   layers.Dense(neuron, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
          layers.Dense(2, activation="softmax"),
      ]
  )

  print(model.summary())


  
  ### compile the model configuration
  model.compile(
      loss = keras.losses.SparseCategoricalCrossentropy(), 
      # if use CategoricalCrossentropy, then the output should be one-hot encoded
      optimizer = keras.optimizers.Adamax(learning_rate=0.001),
      metrics=["accuracy"] # the metrics keras will keep tracking during the training (tf.keras.metrics.AUC())
  )

  ### fit the data
  # callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True), validation_data=(X_test, Y_test),callbacks = [callback]
  history = model.fit(X, Y, batch_size=128, epochs=epoch, verbose=2)

  return model

In [21]:
model_ifeature = individual_ifeature(X_ifeature_train, y_ifeature_train, 257, 10)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 257)               528649    
                                                                 
 dropout_2 (Dropout)         (None, 257)               0         
                                                                 
 dense_5 (Dense)             (None, 2)                 516       
                                                                 
Total params: 529,165
Trainable params: 529,165
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
681/681 - 3s - loss: 1.3697 - accuracy: 0.6906 - 3s/epoch - 5ms/step
Epoch 2/10
681/681 - 3s - loss: 0.6791 - accuracy: 0.7107 - 3s/epoch - 4ms/step
Epoch 3/10
681/681 - 3s - loss: 0.6014 - accuracy: 0.7144 - 3s/epoch - 4ms/step
Epoch 4/10
681/681 - 3s - loss: 0.5865 - accuracy: 0.7178 - 3s/epoc

In [22]:
ifeature_all_label, ifeature_all_label_number, ifeature_report = test_evaluation(model_ifeature, X_ifeature_test, y_ifeature_test)

169/169 - 0s - 432ms/epoch - 3ms/step
              precision    recall  f1-score   support

           0      0.781     0.721     0.750     12846
           1      0.629     0.700     0.663      8689

    accuracy                          0.713     21535
   macro avg      0.705     0.711     0.706     21535
weighted avg      0.720     0.713     0.715     21535

AUROC: 0.7107142362474942


In [25]:
state="train1_ifeature"


lines = ifeature_report
auroc = str(0.7107142362474942)

with open('Result/class_report_%s.txt'%(state), 'w') as f:
    for line in lines:
        f.write(line)
    f.write(auroc)